In [1]:
lines=[]
for i in range(17):
    if i < 10:
        j = '0'+str(i)
    else:
        j = str(i)
    f = open('../../alert.full.maccdc2012_000'+j+'.pcap','r')
    lines.append(f.readlines())
    print(len(lines[i]))

166491
54170
12033
6708
68683
0
82597
90417
57248
450417
526293
402550
171677
25994
152808
350537
131799


In [2]:
g = open('../../alert.full.maccdc2012_00000.pcap','r')
len(g.readlines())

166491

In [3]:
for j in range(17):
    print(j)
    for i in range(1,20):
        if j != 5:
            print(lines[j][-i])        

0
03/16-08:30:07.770000  [**] [1:2013659:3] ET POLICY Self Signed SSL Certificate (SomeOrganizationalUnit) [**] [Classification: Potential Corporate Privacy Violation] [Priority: 1] {TCP} 192.168.27.253:443 -> 192.168.202.110:49653

03/16-08:30:07.760000  [**] [1:2012088:1] ET SHELLCODE Possible Call with No Offset TCP Shellcode [**] [Classification: Executable Code was Detected] [Priority: 1] {TCP} 192.168.202.68:8080 -> 192.168.24.100:1038

03/16-08:30:07.690000  [**] [1:2013659:3] ET POLICY Self Signed SSL Certificate (SomeOrganizationalUnit) [**] [Classification: Potential Corporate Privacy Violation] [Priority: 1] {TCP} 192.168.27.253:443 -> 192.168.202.110:49652

03/16-08:30:07.630000  [**] [1:2012086:1] ET SHELLCODE Possible Call with No Offset TCP Shellcode [**] [Classification: Executable Code was Detected] [Priority: 1] {TCP} 192.168.202.94:443 -> 192.168.24.100:1039

03/16-08:30:07.610000  [**] [1:2101390:8] GPL SHELLCODE x86 inc ebx NOOP [**] [Classification: Executable Cod

We are looking to find attributes to match to the main log. Here we will use the ip addresses and port numbers as well as the time. From the above, the line containing this data all start with '03/', while it looks like no other lines start the same. As such I will use this to access this data from here. We will now look to filter the lines and then look to format each line into a table. We use the same reasoning for finding the attack type and '[class'.

In [4]:
filtered = []
desc = []
for j in range(17):
    if j != 5:
        for i in range(0,len(lines[j])-1):
            if lines[j][i].startswith('03/'):
                filtered.append(lines[j][i])
            if lines[j][i].startswith('[Class'):
                desc.append(lines[j][i])

In [5]:
print(len(filtered))
print(len(desc))
s = 0
for j in range(17):
    s = s + len(lines[j])
print(s)
print(s/len(filtered))

2750406
0
2750422
1.0000058173229698


Here there are approximately 7 lines for each line that starts with '03/' which looking at the data looks like a good approximation. As such, I am reasonably confident that it has picked up all the lines it needed to.  

In [6]:
day=[]
hour=[]
minute=[]
seconds=[]
milli=[]
srcIP=[]
srcPort=[]
dstIP=[]
dstPort=[]
IPv4=[]
Port=[]
num1=[]
num2=[]
num3=[]
atcktype=[]
for i in range(0,len(filtered)):
    y = desc[i].find(']')
    atcktype.append(desc[i][17:y])
    day.append(filtered[i][3:5])
    hour.append(filtered[i][6:8])
    minute.append(filtered[i][9:11])
    seconds.append(filtered[i][12:14])
    milli.append(filtered[i][15:21])
    if filtered[i][25] == '.':
        IPv4.append(True)
        if (filtered[i][22:].find(':') < filtered[i][22:].find('->')) and (filtered[i][22:].find(':') != -1):
            Port.append(True)
            x1 = filtered[i][22:].find(':') + 22
            srcIP.append(filtered[i][22:x1])
            x2 = filtered[i][22:].find('->') + 22
            srcPort.append(filtered[i][x1+1:x2-1])
            x3 = x2 + 3
            x4 = filtered[i][x3:].find(':') + x3
            dstIP.append(filtered[i][x3:x4])
            dstPort.append(filtered[i][x4+1:-1])
        else:
            num1.append(i)
            Port.append(False)
            x1 = filtered[i][22:].find('->') + 22
            srcIP.append(filtered[i][22:x1-1])
            dstIP.append(filtered[i][x1+3:-1])  
            srcPort.append(0)
            dstPort.append(0)
    else:
        IPv4.append(False)
        arrow = filtered[i][22:].find('->') + 22
        if filtered[i][22:arrow].count(':') == 8:
            num2.append(i)
            Port.append(True)
            x1 = filtered[i][22:arrow].rfind(':') + 22
            srcIP.append(filtered[i][22:x1])
            x2 = arrow
            srcPort.append(filtered[i][x1+1:x2-1])
            x3 = x2 + 3
            x4 = filtered[i][x3:].rfind(':') + x3
            dstIP.append(filtered[i][x3:x4])
            dstPort.append(filtered[i][x4+1:-1])
        else:
            num3.append(i)
            Port.append(False)
            x1 = arrow
            srcIP.append(filtered[i][22:x1-1])
            dstIP.append(filtered[i][x1+3:-1])
            srcPort.append(0)
            dstPort.append(0)

IndexError: list index out of range

In [ ]:
print(atcktype[1:10])

In [ ]:
for i in range(5):
    print(filtered[num2[i]])
    print(srcIP[num2[i]])
    print(srcPort[num2[i]])
    print(dstIP[num2[i]])
    print(dstPort[num2[i]])
    print('/')
    print(filtered[num1[i]])
    print(srcIP[num1[i]])
    print(dstIP[num1[i]])
    print('/')
    print(filtered[num3[i]])
    print(srcIP[num3[i]])
    print(dstIP[num3[i]])
    print('/')

The above makes me confident in the ability for the code to separate the relevant information regarless of IPv and if ports are included. We will now format the time. We will use that the first packet in the snort log occurs at the first time instance in conn.log (time = 1331901000.000000).

In [ ]:
time=[]
scaler16 = (int(milli[0]) / 1000000) + int(seconds[0]) + int(minute[0]) * 60 + int(hour[0]) * 3600
scaler17 = (int(milli[1037495]) / 1000000) + int(seconds[1037495]) + int(minute[1037495]) * 60 + int(hour[1037495]) * 3600 #1037495 1175157
for i in range(len(filtered)):
    if int(day[i]) == 16:
        temp = (int(milli[i]) / 1000000) + int(seconds[i]) + int(minute[i]) * 60 + int(hour[i]) * 3600 - scaler16 + 1331901000
    else:
        temp = (int(milli[i]) / 1000000) + int(seconds[i]) + int(minute[i]) * 60 + int(hour[i]) * 3600 - scaler17 +  1331987253.53 #1331987253.53 1331997339.739999
    time.append(str(temp).ljust(17,'0')) 

In [ ]:
for i in range (20,50):
    print(time[-i])
    print(filtered[-i])

In [ ]:
for i in range(1,100):
    print('new')
    print(filtered[-i])
    print(time[-i])
    print(len(filtered) - i)

While looking at how effective my code is at matching to the timings of conn.log I have found couple things:

* There are snort log entries that do not correlate to any entry in conn log, I believe that this is due to conn.log being a subset of the data that the snort log has been formed from. As such, not every snort log has an equivalent in conn.log.

* Some of the packets may be assigned different times as when the packet was recorded it fell between the two discrete times e.g. filtered[1682258] is .85 according to the snort log but is recorded as .84 in conn.log. However, as I believe that this will not be the case for most packets, I will not consider this when matching with the conn.log. I may, if I have time, see how looking at +.01, and -.01 times change the number of packets flagged as malicious in conn.log. 

* While I haven't seen it, given the above point, it is very possible that some of the .xx9999 times in one will be .xx in the other. I will ignore this for the time being, however if I have time I will have the program check for these and see how this affects the number of packets flagged as malicious in conn.log

Overall, I think that the algorithm above works at converting the snort time to the conn.log time. So I will now create a table with all the information and then export this to a table to be compared with conn.log.

In [ ]:
import pandas as pd

In [ ]:
d = {
    'ts' : time,
    'srcIP': srcIP,
    'srcPort': srcPort,
    'dstIP': dstIP,
    'dstPort': dstPort,
    'IPv4': IPv4,
    'Port': Port,
    'type': atcktype
}
df = pd.DataFrame(data=d)
df

In [ ]:
df.to_csv('../data/raw/snort.csv')